In [1]:
!git clone https://github.com/Hotsnown/seminaire-bordeaux-2022.git seminaire &> /dev/null
%pip install nbautoeval &> /dev/null
from seminaire.evaluation.jour3.cleaning.project import remove_html_tags_func, remove_url_func, remove_accented_chars_func, remove_punctuation_func,remove_irr_char_func, remove_extra_whitespaces_func, word_count_func
!git clone https://github.com/chinmusique/outcome-prediction.git &> /dev/null

SyntaxError: ignored

# Nettoyer l'ensemble de données d'identification des résultats

## 2 Importer les bibliothèques et les données

Si vous utilisez la bibliothèque nltk pour la première fois, vous devez importer et télécharger les éléments suivants :

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.


True

In [3]:
import pandas as pd
import numpy as np

import pickle as pk

import warnings
warnings.filterwarnings("ignore")


from bs4 import BeautifulSoup
import unicodedata
import re

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

from nltk.corpus import stopwords


from nltk.corpus import wordnet
from nltk import pos_tag
from nltk import ne_chunk

from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from nltk.probability import FreqDist
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [5]:
import json
import pandas as pd
from pathlib import Path

!git clone https://github.com/chinmusique/outcome-prediction.git

# creating list of json files with full paths
paths = Path(r'/content/outcome-prediction/data/manual_annotation').glob("*.json")

df_list = [] # empty list to store dataframes during the loop
for p in paths:
    data = json.load(open(p))
    data = data["annotations"]
    df_nested_list = pd.json_normalize(
        data,
        meta=['name','description','custom_fields'],)
    df_nested_list['file_name'] = p.name  # creating column to store file name: p.name
    df_list.append(df_nested_list) # dataframe to the list

df_raw = pd.concat(df_list, axis=0, ignore_index=True) # creating One Large Dataframe from all stored in the list
print(df_raw.columns)

df = df_raw.drop(columns=["role",	"class",	"index",	"contentIdx.start",	"contentIdx.end", "_id", "index"])
df.head()

Cloning into 'outcome-prediction'...
remote: Enumerating objects: 584, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 584 (delta 3), reused 0 (delta 0), pack-reused 562
Receiving objects: 100% (584/584), 5.20 MiB | 10.82 MiB/s, done.
Resolving deltas: 100% (11/11), done.
Index(['_id', 'role', 'content', 'class', 'index', 'contentIdx.start',
       'contentIdx.end', 'file_name'],
      dtype='object')


,content,file_name
0,We reverse the orders of the trial court denyi...,77129.json
1,We therefore reverse Defendant's conviction an...,77129.json
2,NaN,77129.json
3,"For the reasons discussed herein, we reverse ...",18357.json
4,"Accordingly, the cause is reversed and remande...",18357.json


Cependant, nous ne travaillerons qu'avec la partie suivante de l'ensemble de données :

Examinons de plus près la première série d'examens :

In [9]:
df = df[["content"]]

df['content'].iloc[0]


"We reverse the orders of the trial court denying Defendant's motion to dismiss the indictment."

In [10]:
df.dtypes


content    object
dtype: object

Pour plus de sécurité, je convertis les avis sous forme de chaînes pour pouvoir les utiliser correctement.

In [12]:
df['content'] = df['content'].astype(str)

### 3 Prétraitement du texte

Il y a quelques étapes de prétraitement du texte à prendre en compte et quelques autres que vous pouvez faire. Dans cet article, je parlerai du nettoyage de texte.

#### 3.1 Nettoyage du texte

Ici, j'ai créé un exemple de chaîne, où vous pouvez très bien comprendre les étapes suivantes.

In [21]:
messy_text = \
"Hi e-v-e-r-y-o-n-e !!!@@@!!! I gave a 5-star rating. \
Bought this special product here: https://www.amazon.com/. Another link: www.amazon.com/ \
Here the HTML-Tag as well:  <a href='https://www.amazon.com/'> …</a>. \
I HIGHLY RECOMMEND THIS PRDUCT !! \
I @ (love) [it] <for> {all} ~it's* |/ #special / ^^characters;! \
I am currently investigating the special device and am excited about the features. Love it! \
Furthermore, I found the support really great. Paid about 180$ for it (5.7inch version, 4.8'' screen). \
Sómě special Áccěntěd těxt and words like résumé, café or exposé.\
"
messy_text

"Hi e-v-e-r-y-o-n-e !!!@@@!!! I gave a 5-star rating. Bought this special product here: https://www.amazon.com/. Another link: www.amazon.com/ Here the HTML-Tag as well:  <a href='https://www.amazon.com/'> …</a>. I HIGHLY RECOMMEND THIS PRDUCT !! I @ (love) [it] <for> {all} ~it's* |/ #special / ^^characters;! I am currently investigating the special device and am excited about the features. Love it! Furthermore, I found the support really great. Paid about 180$ for it (5.7inch version, 4.8'' screen). Sómě special Áccěntěd těxt and words like résumé, café or exposé."

Dans ce qui suit, je vais effectuer les étapes individuelles pour le nettoyage du texte et utiliser toujours des parties de la chaîne messy_text.

#### 3.1.1 Conversion en minuscules

En général, il est conseillé de formater le texte entièrement en minuscules.

In [22]:
messy_text_lower_case = \
"I HIGHLY RECOMMEND THIS PRDUCT !!\
"
messy_text_lower_case

'I HIGHLY RECOMMEND THIS PRDUCT !!'

In [ ]:
#TODO mettez messy_text_lower_case en minuscule


'i highly recommend this prduct !!'

#### 3.1.2 Suppression des balises HTML

In [24]:
messy_text_html = \
"Here the HTML-Tag as well:  <a href='https://www.amazon.com/'> …</a>.\
"
messy_text_html

"Here the HTML-Tag as well:  <a href='https://www.amazon.com/'> …</a>."

In [ ]:
#TODO : créez une fonction supprimer_html(text) qui supprime les balises html. 
#Si vous n'y arrivez pas, vous pouvez utiliser la fonction remove_html_tags_func.

In [26]:
#remove_html_tags_func(messy_text_html)

'Here the HTML-Tag as well:   ….'

#### 3.1.3 Suppression d'URL

In [27]:
messy_text_url = \
"Bought this product here: https://www.amazon.com/. Another link: www.amazon.com/\
"
messy_text_url

'Bought this product here: https://www.amazon.com/. Another link: www.amazon.com/'

In [28]:
#TODO : créez une fonction supprimer_url(text) qui supprime les urls. 
#Si vous n'y arrivez pas, vous pouvez utiliser la fonction remove_url_func.

In [29]:
#remove_url_func(messy_text_url)


'Bought this product here:  Another link: '

#### 3.1.4 Suppression des caractères accentués

In [30]:
messy_text_accented_chars = \
"Sómě Áccěntěd těxt and words like résumé, café or exposé.\
"
messy_text_accented_chars

'Sómě Áccěntěd těxt and words like résumé, café or exposé.'

In [31]:
#TODO : créez une fonction supprimer_accent(text) qui supprime les accents. 
#Si vous n'y arrivez pas, vous pouvez utiliser la fonction remove_accented_chars_func.

In [32]:
#remove_accented_chars_func(messy_text_accented_chars)


'Some Accented text and words like resume, cafe or expose.'

#### 3.1.5 Supprimer la ponctuation

Punctuation is essentially the following set of symbols: [!”#$%&’()*+,-./:;<=>?@[]^_`{|}~]

In [33]:
messy_text_remove_punctuation = \
"Furthermore, I found the support really great. Paid about 180$ for it (5.7inch version, 4.8'' screen).\
"
messy_text_remove_punctuation

"Furthermore, I found the support really great. Paid about 180$ for it (5.7inch version, 4.8'' screen)."

In [34]:
#TODO : créez une fonction supprimer_ponctuation(text) qui supprime les accents. 
#Si vous n'y arrivez pas, vous pouvez utiliser la fonction remove_accented_chars_func.

In [35]:
#remove_punctuation_func(messy_text_remove_punctuation)


'Furthermore  I found the support really great  Paid about 180  for it  5 7inch version  4 8   screen  '

#### 3.1.6 Suppression des caractères non pertinents (chiffres et ponctuation)

In [36]:
messy_text_irr_char = \
"Furthermore, I found the support really great. Paid about 180$ for it (5.7inch version, 4.8'' screen).\
"
messy_text_irr_char

"Furthermore, I found the support really great. Paid about 180$ for it (5.7inch version, 4.8'' screen)."

In [37]:
#TODO : créez une fonction supprimer_caractères_spéciaux(text) qui supprime les caractères_spéciaux. 
#Si vous n'y arrivez pas, vous pouvez utiliser la fonction remove_irr_char_func.

In [38]:
#remove_irr_char_func(messy_text_irr_char)


'Furthermore  I found the support really great  Paid about      for it     inch version        screen  '

#### 3.1.7 Suppression des espaces blancs supplémentaires

In [39]:
messy_text_extra_whitespaces = \
"I  am   a  text    with  many   whitespaces.\
"
messy_text_extra_whitespaces

'I  am   a  text    with  many   whitespaces.'

In [40]:
#TODO : créez une fonction supprimer_extra_espaces(text) qui supprime les doubles espaces. 
#Si vous n'y arrivez pas, vous pouvez utiliser la fonction remove_extra_whitespaces_func.

In [41]:
#remove_extra_whitespaces_func(messy_text_extra_whitespaces)

'I am a text with many whitespaces.'

Cette fonction est utile à la fin, pour corriger les problèmes de formattage causés par la suppression de la ponctuation ou des mots vides.

#### 3.1.8 Extra : Compter les mots

Il est intéressant d'afficher le nombre de mots existants, notamment pour la validation des étapes de pré-traitement. Nous utiliserons cette fonction encore et encore dans les étapes ultérieures.

In [42]:
messy_text_word_count = \
"How many words do you think I will contain?\
"
messy_text_word_count

'How many words do you think I will contain?'

In [43]:
#TODO : créez une fonction compter_mots(text) qui compte les mots. 
#Si vous n'y arrivez pas, vous pouvez utiliser la fonction word_count_func.

In [44]:
#word_count_func(messy_text_word_count)

9

#### 3.1.10 Application à l'exemple de chaîne

Avant cela, j'ai utilisé des modules de texte individuels pour montrer comment fonctionnent toutes les étapes de nettoyage de texte. Il est maintenant temps d'appliquer ces fonctions à Example String (et ensuite au DataFrame) l'une après l'autre.

In [48]:
messy_text


"Hi e-v-e-r-y-o-n-e !!!@@@!!! I gave a 5-star rating. Bought this special product here: https://www.amazon.com/. Another link: www.amazon.com/ Here the HTML-Tag as well:  <a href='https://www.amazon.com/'> …</a>. I HIGHLY RECOMMEND THIS PRDUCT !! I @ (love) [it] <for> {all} ~it's* |/ #special / ^^characters;! I am currently investigating the special device and am excited about the features. Love it! Furthermore, I found the support really great. Paid about 180$ for it (5.7inch version, 4.8'' screen). Sómě special Áccěntěd těxt and words like résumé, café or exposé."

In [49]:
messy_text_lower = #TODO
messy_text_lower

"hi e-v-e-r-y-o-n-e !!!@@@!!! i gave a 5-star rating. bought this special product here: https://www.amazon.com/. another link: www.amazon.com/ here the html-tag as well:  <a href='https://www.amazon.com/'> …</a>. i highly recommend this prduct !! i @ (love) [it] <for> {all} ~it's* |/ #special / ^^characters;! i am currently investigating the special device and am excited about the features. love it! furthermore, i found the support really great. paid about 180$ for it (5.7inch version, 4.8'' screen). sómě special áccěntěd těxt and words like résumé, café or exposé."

In [50]:
messy_text_wo_html = #TODO
messy_text_wo_html

"hi e-v-e-r-y-o-n-e !!!@@@!!! i gave a 5-star rating. bought this special product here: https://www.amazon.com/. another link: www.amazon.com/ here the html-tag as well:   …. i highly recommend this prduct !! i @ (love) [it]  {all} ~it's* |/ #special / ^^characters;! i am currently investigating the special device and am excited about the features. love it! furthermore, i found the support really great. paid about 180$ for it (5.7inch version, 4.8'' screen). sómě special áccěntěd těxt and words like résumé, café or exposé."

In [51]:
messy_text_wo_url = #TODO
messy_text_wo_url

"hi e-v-e-r-y-o-n-e !!!@@@!!! i gave a 5-star rating. bought this special product here:  another link:  here the html-tag as well:   …. i highly recommend this prduct !! i @ (love) [it]  {all} ~it's* |/ #special / ^^characters;! i am currently investigating the special device and am excited about the features. love it! furthermore, i found the support really great. paid about 180$ for it (5.7inch version, 4.8'' screen). sómě special áccěntěd těxt and words like résumé, café or exposé."

In [52]:
messy_text_wo_acc_chars = #TODO
messy_text_wo_acc_chars

"hi e-v-e-r-y-o-n-e !!!@@@!!! i gave a 5-star rating. bought this special product here:  another link:  here the html-tag as well:   .... i highly recommend this prduct !! i @ (love) [it]  {all} ~it's* |/ #special / ^^characters;! i am currently investigating the special device and am excited about the features. love it! furthermore, i found the support really great. paid about 180$ for it (5.7inch version, 4.8'' screen). some special accented text and words like resume, cafe or expose."

In [53]:
messy_text_wo_punct = #TODO
messy_text_wo_punct

'hi e v e r y o n e           i gave a 5 star rating  bought this special product here   another link   here the html tag as well         i highly recommend this prduct    i    love   it    all   it s      special     characters   i am currently investigating the special device and am excited about the features  love it  furthermore  i found the support really great  paid about 180  for it  5 7inch version  4 8   screen   some special accented text and words like resume  cafe or expose '

In [54]:
messy_text_wo_irr_char = #TODO
messy_text_wo_irr_char

'hi e v e r y o n e           i gave a   star rating  bought this special product here   another link   here the html tag as well         i highly recommend this prduct    i    love   it    all   it s      special     characters   i am currently investigating the special device and am excited about the features  love it  furthermore  i found the support really great  paid about      for it     inch version        screen   some special accented text and words like resume  cafe or expose '

In [55]:
clean_text = #TODO
clean_text

'hi e v e r y o n e i gave a star rating bought this special product here another link here the html tag as well i highly recommend this prduct i love it all it s special characters i am currently investigating the special device and am excited about the features love it furthermore i found the support really great paid about for it inch version screen some special accented text and words like resume cafe or expose'

In [56]:
print('Nombre de mots: ' + str(word_count_func(clean_text)))


Number of words: 80


#### 3.1.11 Application au DataFrame

Maintenant, nous appliquons les étapes de nettoyage de texte indiquées ci-dessus au DataFrame :

In [57]:
df.head()


,content
0,We reverse the orders of the trial court denyi...
1,We therefore reverse Defendant's conviction an...
2,nan
3,"For the reasons discussed herein, we reverse ..."
4,"Accordingly, the cause is reversed and remande..."


In [59]:
df['clean_content'] = df['content'].str.lower()
df['clean_content'] = df['clean_content'].apply(#TODO)
df['clean_content'] = df['clean_content'].apply(#TODO)
df['clean_content'] = df['clean_content'].apply(#TODO)
df['clean_content'] = df['clean_content'].apply(#TODO)
df['clean_content'] = df['clean_content'].apply(#TODO)
df['clean_content'] = df['clean_content'].apply(#TODO)

df.head()

,content,clean_content
0,We reverse the orders of the trial court denyi...,we reverse the orders of the trial court denyi...
1,We therefore reverse Defendant's conviction an...,we therefore reverse defendant s conviction an...
2,nan,nan
3,"For the reasons discussed herein, we reverse ...",for the reasons discussed herein we reverse an...
4,"Accordingly, the cause is reversed and remande...",accordingly the cause is reversed and remanded...


Let’s now compare the sentences from line 1 with the ones we have now edited:



In [63]:
df['content'].iloc[0]

"We reverse the orders of the trial court denying Defendant's motion to dismiss the indictment."

In [62]:
df['clean_content'].iloc[0]


'we reverse the orders of the trial court denying defendant s motion to dismiss the indictment'

Enfin, nous sortons le nombre de mots et les stockons dans une colonne séparée. De cette façon, nous pouvons voir si et dans quelle mesure le nombre de mots a changé dans les étapes suivantes.

In [66]:
df['Word_Count'] = df['clean_content'].apply(word_count_func)

df[['clean_content', 'Word_Count']].head()

,clean_content,Word_Count
0,we reverse the orders of the trial court denyi...,16
1,we therefore reverse defendant s conviction an...,15
2,nan,1
3,for the reasons discussed herein we reverse an...,12
4,accordingly the cause is reversed and remanded...,14


Voici le nombre moyen de mots :

In [67]:
print('Average of words counted: ' + str(df['Word_Count'].mean()))


Average of words counted: 15.227422907488986


1. Jour 1
    * Variables
        * [exercice1](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/1.%20variables/1.1%20helloworld.ipynb)
        * [exercice2](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/1.%20variables/1.2%20%C3%A9viter%20les%20errreurs%20de%20nommage.ipynb)
        * [mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/1.%20variables/1.3%20mini-project.ipynb#scrollTo=RPB2xCMdA6lV)
    * Strings
        * [exercice1](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/2.%20strings/2.1%20concat.ipynb)
        * [exercice2](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/2.%20strings/2.2%20string_methods.ipynb)
        * [mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/2.%20strings/2.3%20mini-project.ipynb)
    * Opérations
        * [exercice1](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/3.%20operations/3.1%20math.ipynb)
        * [exercice2](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/3.%20operations/3.2%20bool%C3%A9en.ipynb)
        * [mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/3.%20operations/3.3%20mini-project.ipynb)

2. Jour 2
    * Listes
        * [Définition](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.1%20Listes/2.1.1%20d%C3%A9finition%20liste.ipynb?hl=fr)
        * [Strings et listes](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.1%20Listes/2.1.2%20String%20as%20list%20of%20characters.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.1%20Listes/2.1.3%20mini-project.ipynb?hl=fr)
    * Fonctions
        * [Définition I](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.2%20Fonctions/2.2.1%20d%C3%A9finition%20fonctions.ipynb?hl=fr)
        * [Définition II](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.2%20Fonctions/2.2.2%20scope%20et%20fonctions%20imbriqu%C3%A9es.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.2%20Fonctions/2.2.3%20mini-project.ipynb?hl=fr) 
    * Librairies
        * [Pandas](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.3%20Librairies/2.3.1%20Pandas.ipynb?hl=fr)
        * [Matplotlib](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.3%20Librairies/2.3.2%20Matplotlib.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.3%20Librairies/2.3.3%20mini-project.ipynb?hl=fr) 
3. Jour 3
    * Introduction à la NLP
        * [Charger des un corpus](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.1%20what%20is%20nlp/3.1.1%20Accessing%20Text.ipynb?hl=fr)
        * [Traitement de texte dans Pandas](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.1%20what%20is%20nlp/3.1.2%20Working%20with%20text%20data%20in%20pandas.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.1%20what%20is%20nlp/3.1.3%20mini-project.ipynb?hl=fr)
    * Segmentation
        * [Segmentation de tokens](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.2%20Segmentation/3.2.1%20Token%20segmentation.ipynb?hl=fr)
        * [Segmentation de phrase](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.2%20Segmentation/3.2.2%20Sentence%20segmentation.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.2%20Segmentation/3.2.3%20mini-project.ipynb?hl=fr)
    * Nettoyage de texte
        * [Stopwords](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.3%20text%20cleaning.ipynb/3.3.1%20stopwords.ipynb?hl=fr)
        * [Normalisation](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.3%20text%20cleaning.ipynb/3.3.2%20Normalizing%20Text.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.3%20text%20cleaning.ipynb/3.3.3%20mini-project.ipynb?hl=fr)
4. Jour 4
    * Apprentissage supervisé
        * [Régression linéaire](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.1%20supervised%20learning/4.3.1%20linear%20regression.ipynb?hl=fr)
        * [Evaluation](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.1%20supervised%20learning/4.3.2%20evaluale.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.1%20supervised%20learning/4.3.3%20mini-project.ipynb?hl=fr)
    * Pré-traitement de texte
        * [Featurization de textes](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.2%20text%20preprocessing/4.2.1%20text%20featurization.ipynb?hl=fr)
        * [Featurization de labels](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.2%20text%20preprocessing/4.2.2%20label%20featurization.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.2%20text%20preprocessing/4.2.3%20mini-project.ipynb?hl=fr)
    * Classification de texte
        * [EDA](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.3%20text%20classification/4.1.1%20EDA.ipynb?hl=fr)
        * [Apprentissage supervisé textuel](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.3%20text%20classification/4.1.1%20EDA.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.3%20text%20classification/4.1.3%20mini-project.ipynb?hl=fr)
5. Jour 5
    * [Projet final](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour5/final-project.ipynb?hl=fr)